# Lab 2: 診断エージェント

## 概要
CRM アプリケーションスタックの CloudWatch ログとメトリクスを分析する Strands ベースの診断エージェントを構築します。

## 目標
- インシデント分析のための Strands エージェントを作成
- CloudWatch ログとメトリクス取得用のカスタムツールを実装
- EC2、DynamoDB、NGINX ログを取得するツールを構築
- 実際のアプリケーションログに対してエージェントをテスト
- 診断精度を検証

## 学習内容
- ツール使用を伴う Strands エージェントの構築方法
- AWS Lambda バックエンドツールの作成方法
- CloudWatch ログのプログラムによる分析方法
- エージェントの診断ワークフローと推論

## アーキテクチャ概要

```
┌─────────────────┐         ┌──────────────────────┐         ┌─────────────────────┐
│  ユーザーリクエスト │────────▶│   Strands Agent      │────────▶│  AgentCore Gateway  │
│  （診断          │         │   （診断）            │         │  （MCP プロトコル）   │
│   クエリ）        │         └──────────────────────┘         └─────────────────────┘
└─────────────────┘                    │                               │
                                       │                               │
                                       ▼                               ▼
                          ┌────────────────────────┐      ┌──────────────────────┐
                          │  診断ツール             │      │  Lambda 関数          │
                          │  ├─ EC2 ログ           │      │  （ZIP デプロイ）      │
                          │  ├─ NGINX ログ         │      │  ├─ セッション管理     │
                          │  ├─ DynamoDB メトリクス │      │  ├─ ツール実行        │
                          │  └─ CloudWatch CPU/    │      │  └─ エラー処理        │
                          │     メモリメトリクス    │      └──────────────────────┘
                          └────────────────────────┘                │
                                       │                             │
                                       ▼                             ▼
                          ┌────────────────────────┐      ┌──────────────────────┐
                          │  AWS サービス          │◀─────│  分析結果            │
                          │  ├─ CloudWatch Logs    │      │  と検証              │
                          │  ├─ EC2 インスタンス    │      └──────────────────────┘
                          │  ├─ DynamoDB テーブル   │
                          │  └─ CloudWatch Metrics │
                          └────────────────────────┘
```

### 主要コンポーネント:

• **マルチツールオーケストレーション**: エージェントが7つの診断ツール（EC2、NGINX、DynamoDB、CloudWatch）を調整
• **セキュアな実行**: VPC アクセスと IAM ベース認証を持つ Lambda 関数
• **リアルタイム分析**: ライブログとメトリクス取得のための CloudWatch 統合

## 0. 必要なパッケージのインストール

このセルを最初に実行して、すべての依存関係がインストールされていることを確認してください。

In [ ]:
%pip install -q -r requirements.txt
print("✅ ワークショップの依存関係がインストールされました")

## 1. 必要なモジュールのインポート

In [ ]:
# AWS SDK と設定
import boto3
import json
import datetime
from pprint import pprint

# ワークショップ設定
from lab_helpers.config import MODEL_ID, AWS_REGION, AWS_PROFILE
from lab_helpers.constants import PARAMETER_PATHS

# AWS クライアントを初期化
cloudwatch_client = boto3.client('logs', region_name=AWS_REGION)
ec2_client = boto3.client('ec2', region_name=AWS_REGION)
lambda_client = boto3.client('lambda', region_name=AWS_REGION)
sts_client = boto3.client('sts', region_name=AWS_REGION)
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

from lab_helpers.lab_01.fault_injection import initialize_fault_injection
from lab_helpers.parameter_store import put_parameter, get_parameter

# AWS クライアントを初期化し、SSM からインフラストラクチャリソース ID を取得
print("障害注入ユーティリティを初期化中...")
resources = initialize_fault_injection(AWS_REGION, AWS_PROFILE)

print(f"\n検出されたインフラストラクチャリソース:")

print(f"  Nginxインスタンス: {resources.get('nginx_instance_id', 'Not found')}")
print(f"  アプリインスタンス: {resources.get('app_instance_id', 'Not found')}")
print(f"  CRM Activitiesテーブル: {resources.get('crm_activities_table_name', 'Not found')}")
print(f"  CRM Customersテーブル: {resources.get('crm_customers_table_name', 'Not found')}")
print(f"  CRM Dealsテーブル: {resources.get('crm_deals_table_name', 'Not found')}")

print("✅ インポートが読み込まれました")

## 2. 前提条件の確認

In [ ]:
# 前提条件が利用可能か確認
try:
    identity = sts_client.get_caller_identity()
    account_id = identity['Account']
    
    import bedrock_agentcore
    from strands import Agent
    
    print(f"✅ 前提条件を確認しました: AWSアカウント {account_id}、bedrock-agentcore + strandsが利用可能です")
except Exception as e:
    print(f"❌ エラー: {e}")
    print("AWS認証情報が設定され、すべてのパッケージがインストールされていることを確認してください。")

## 3. ヘルパーツールの作成

In [ ]:
from lab_helpers import mock_data

# CloudWatch からログとメトリクスを取得するヘルパーツール関数


def fetch_crm_app_logs(log_group_name="/aws/sre-workshop/crm-application", hours=2, use_mock=False):
    """CloudWatch から CRM アプリケーションログを取得"""
    if use_mock:
        return mock_data.get_ec2_logs()
    
    try:
        now = datetime.datetime.now(datetime.timezone.utc)
        start_time = int((now - datetime.timedelta(hours=hours)).timestamp() * 1000)
        end_time = int(now.timestamp() * 1000)
        
        response = cloudwatch_client.filter_log_events(
            logGroupName=log_group_name,
            startTime=start_time,
            endTime=end_time,
            filterPattern='?error ?throttle',
            limit=500
        )
        return response.get('events', [])
    except Exception as e:
        return [{"message": f"Error fetching EC2 logs: {str(e)}"}]
        

def fetch_ec2_logs(log_group_name="/aws/sre-workshop/application", hours=2, use_mock=False):
    """CloudWatch から EC2 アプリケーションログを取得"""
    if use_mock:
        return mock_data.get_ec2_logs()
    
    try:
        now = datetime.datetime.now(datetime.timezone.utc)
        start_time = int((now - datetime.timedelta(hours=hours)).timestamp() * 1000)
        end_time = int(now.timestamp() * 1000)
        
        response = cloudwatch_client.filter_log_events(
            logGroupName=log_group_name,
            startTime=start_time,
            endTime=end_time,
            filterPattern='?error ?throttle',
            limit=500
        )
        return response.get('events', [])
    except Exception as e:
        return [{"message": f"Error fetching EC2 logs: {str(e)}"}]

def fetch_nginx_error_logs(log_group_name="/aws/sre-workshop/nginx/error", hours=2, use_mock=False):
    """CloudWatch から NGINX エラーログを取得"""
    if use_mock:
        return mock_data.get_nginx_logs()
    
    try:
        now = datetime.datetime.now(datetime.timezone.utc)
        start_time = int((now - datetime.timedelta(hours=hours)).timestamp() * 1000)
        end_time = int(now.timestamp() * 1000)
        
        response = cloudwatch_client.filter_log_events(
            logGroupName=log_group_name,
            startTime=start_time,
            endTime=end_time,
            filterPattern='?error ?throttle',
            limit=500
        )
        return response.get('events', [])
    except Exception as e:
        return [{"message": f"Error fetching NGINX error logs: {str(e)}"}]

def fetch_nginx_access_logs(log_group_name="/aws/sre-workshop/nginx/access", hours=24, use_mock=False):
    """CloudWatch から NGINX アクセス/エラーログを取得"""
    if use_mock:
        return mock_data.get_nginx_logs()
    
    try:
        now = datetime.datetime.now(datetime.timezone.utc)
        start_time = int((now - datetime.timedelta(hours=hours)).timestamp() * 1000)
        end_time = int(now.timestamp() * 1000)
        
        response = cloudwatch_client.filter_log_events(
            logGroupName=log_group_name,
            startTime=start_time,
            endTime=end_time,
            limit=100
        )
        return response.get('events', [])
    except Exception as e:
        return [{"message": f"Error fetching NGINX access logs: {str(e)}"}]

def fetch_dynamodb_metrics(table_name, period_minutes=60, use_mock=False):
    """CloudWatch から DynamoDB オペレーションログを取得"""
    if use_mock:
        return mock_data.get_dynamodb_logs()
    
    try:
        end_time = datetime.datetime.utcnow()
        start_time = end_time - datetime.timedelta(minutes=period_minutes)
    
    # get_metric_data を使用して1回の呼び出しですべてのメトリクスをクエリ
        cloudwatch = boto3.client('cloudwatch', region_name=AWS_REGION)
        response = cloudwatch.get_metric_data(
        MetricDataQueries=[
            {
                'Id': 'read_capacity',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'ConsumedReadCapacityUnits',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Sum'
                }
            },
            {
                'Id': 'write_capacity',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'ConsumedWriteCapacityUnits',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Sum'
                }
            },
            {
                'Id': 'throttled',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'ThrottledRequests',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Sum'
                }
            },
            {
                'Id': 'user_errors',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'UserErrors',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Sum'
                }
            },
            {
                'Id': 'system_errors',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'SystemErrors',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Sum'
                }
            },
            {
                'Id': 'latency',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/DynamoDB',
                        'MetricName': 'SuccessfulRequestLatency',
                        'Dimensions': [{'Name': 'TableName', 'Value': table_name}]
                    },
                    'Period': 300,
                    'Stat': 'Average'
                }
            }
        ],
        StartTime=start_time,
        EndTime=end_time
    )
    
    # レスポンスから値を抽出
        result = {
        'table_name': table_name,
        'timestamp': end_time.isoformat(),
        'read_capacity': 0,
        'write_capacity': 0,
        'throttled_requests': 0,
        'user_errors': 0,
        'system_errors': 0,
        'avg_latency_ms': None
    }
    
        for metric_result in response['MetricDataResults']:
            metric_id = metric_result['Id']
            values = metric_result['Values']
        
            if values:
                if metric_id == 'latency':
                    result['avg_latency_ms'] = sum(values) / len(values)
            else:
                result[metric_id.replace('_', '_')] = sum(values)
    
        return result
    except Exception as e:
        return [{"message": f"Error fetching DynamoDB logs: {str(e)}"}]



def get_cpu_metrics(instance_id, period_minutes=60):
    """CloudWatch メトリクスを取得するヘルパー関数"""
    cloudwatch = boto3.client('cloudwatch', region_name=AWS_REGION)
    end_time = datetime.datetime.now(datetime.UTC)
    start_time = end_time - datetime.timedelta(minutes=period_minutes)

    
    response = cloudwatch.get_metric_data(
        MetricDataQueries=[{
            'Id': 'm1',
            'MetricStat': {
                'Metric': {
                    'Namespace': 'AWS/EC2',
                    'MetricName': 'CPUUtilization',
                    'Dimensions': [{'Name': 'InstanceId', 'Value': instance_id}]
                },
                'Period': 60,
                'Stat': 'Average'
            }
        }],
        StartTime=start_time,
        EndTime=end_time
    )
    
    values = response['MetricDataResults'][0]['Values']
    return values[-1] if values else None

def get_memory_metrics(instance_id, period_minutes=60):
    """CloudWatch メトリクスを取得するヘルパー関数"""
    cloudwatch = boto3.client('cloudwatch', region_name=AWS_REGION)
    end_time = datetime.datetime.now(datetime.UTC)

    start_time = end_time - datetime.timedelta(minutes=period_minutes)

    
    response = cloudwatch.get_metric_data(
        MetricDataQueries=[{
            'Id': 'm1',
            'MetricStat': {
                'Metric': {
                    'Namespace': 'AWS/EC2',
                    'MetricName': 'mem_used_percent',
                    'Dimensions': [{'Name': 'InstanceId', 'Value': instance_id}]
                },
                'Period': 60,
                'Stat': 'Average'
            }
        }],
        StartTime=start_time,
        EndTime=end_time
    )
    
    values = response['MetricDataResults'][0]['Values']
    return values[-1] if values else None



print("✅ ヘルパーツールが定義されました")

## 4. ヘルパーツールのテスト

In [ ]:
# セクション 3 で作成したヘルパーツールをテスト

# 1. アプリケーションリソースを取得
nginx_instance_id = resources.get('nginx_instance_id')
app_instance_id = resources.get('app_instance_id')
crm_activities_table_name = resources.get('crm_activities_table_name')
crm_customers_table_name = resources.get('crm_customers_table_name')
crm_deals_table_name = resources.get('crm_deals_table_name')

# 2. いくつかの値でツールをテスト
crm_app_logs=fetch_crm_app_logs()
print(crm_app_logs)
ec2_logs = fetch_ec2_logs()
print(ec2_logs)
nginx_error_logs = fetch_nginx_error_logs()
print(nginx_error_logs)
nginx_access_logs = fetch_nginx_access_logs()
print(nginx_access_logs)
ddb_logs = fetch_dynamodb_metrics(table_name=crm_customers_table_name)

print(ddb_logs)

cpu_metrics=get_cpu_metrics(instance_id=nginx_instance_id)
print(cpu_metrics)
memory_metrics=get_memory_metrics(instance_id=nginx_instance_id)
print(memory_metrics)

#print(f"✅ ヘルパーツールを確認: EC2({len(ec2_logs)}), NGINX({len(nginx_logs)}), DynamoDB({len(ddb_logs)}), CPU({len(cpu_metrics)}), Memory({len(memory_metrics)})")

## 5. Strands フレームワークをローカルに追加

**目標:** 診断ツールを Strands と統合し、インテリジェントなエージェントベースの推論を実現します。

**アプローチ:** Tool オブジェクトを定義し、エージェントインスタンスを作成し、ローカルでエンドツーエンドテストを実行します。

**重要な学習ポイント:** Strands が複雑な診断のためにツール使用をどのようにオーケストレーションするか。

In [ ]:
### 5.1: Strands ツールを定義

from strands import tool

table_names = [key for key in resources.keys() if key.endswith('_table_name') and 'crm' in key]
nginx_instance_id = resources.get('nginx_instance_id')
app_instance_id = resources.get('app_instance_id')
crm_activities_table_name = resources.get('crm_activities_table_name')
crm_customers_table_name = resources.get('crm_customers_table_name')
crm_deals_table_name = resources.get('crm_deals_table_name')


# ツール 1: CRM アプリログ
@tool(description="Fetch CRM application logs to identify application errors and issues")
def get_crm_app_logs(limit: int = 10):
    """最新の CRM アプリケーションログを取得"""
    crm_app_logs = fetch_crm_app_logs()
    return crm_app_logs

# ツール 2: EC2 ログ
@tool(description="Fetch EC2 application logs to identify application errors and issues")
def get_ec2_logs(limit: int = 10):
    """最新の EC2 アプリケーションログを取得"""
    ec2_logs = fetch_ec2_logs()
    return ec2_logs

# ツール 3: NGINX エラーログ
@tool(description="Fetch NGINX error logs")
def get_nginx_error_logs():
    """NGINX エラーログを取得"""
    nginx_error_logs = fetch_nginx_error_logs()
    return nginx_error_logs

# ツール 4: NGINX アクセスログ
@tool(description="Fetch NGINX access logs")
def get_nginx_access_logs():
    """NGINX アクセス/エラーログを取得"""
    nginx_access_logs = fetch_nginx_access_logs()
    return nginx_access_logs

# ツール 5: DynamoDB メトリクス
@tool(description="Fetch DynamoDB metrics to detect throttling and service issues")
def get_dynamodb_metrics():
    """DynamoDB オペレーションメトリクスを取得"""
    ddb_metrics=""
    for table in table_names:
        ddb_metrics+=str(fetch_dynamodb_metrics(table_name=table))
      
    return ddb_metrics

# ツール 6: アプリケーション CPU メトリクス
@tool(description="Fetch CloudWatch metrics (CPU) to analyze resource utilization for an instance")
def get_cloudwatch_cpu_metrics():
    """CloudWatch CPU メトリクスを取得"""
    cpu_metrics=get_cpu_metrics(instance_id=nginx_instance_id) + get_cpu_metrics(instance_id=app_instance_id)
    return cpu_metrics

# ツール 7: アプリケーションメモリメトリクス
@tool(description="Fetch CloudWatch metrics (memory) to analyze resource utilization an instance")
def get_cloudwatch_memory_metrics():
    """CloudWatch メモリメトリクスを取得"""
    memory_metrics=get_memory_metrics(instance_id=nginx_instance_id) + get_memory_metrics(instance_id=app_instance_id)
    return memory_metrics

print("✅ Strandsツールが定義されました（7ツール）")
print("   • CRMログ取得")
print("   • EC2ログ取得")
print("   • NGINXエラーログ取得") 
print("   • NGINXアクセスログ取得")
print("   • DynamoDBメトリクス取得")
print("   • CloudWatch CPUメトリクス取得")
print("   • CloudWatch メモリメトリクス取得")

In [ ]:
### 5.2: Strands エージェントを作成

from strands import Agent
from lab_helpers.config import MODEL_ID

# 診断ツールを持つエージェントを作成
diagnostic_agent = Agent(
    name="system_diagnostics_agent",
    description="Expert system diagnostics agent for analyzing logs and metrics",
    model=MODEL_ID,
    tools=[
        get_crm_app_logs,
        get_ec2_logs,
        get_nginx_error_logs,
        get_nginx_access_logs,
        get_dynamodb_metrics,
        get_cloudwatch_cpu_metrics,
        get_cloudwatch_memory_metrics
    ],
    system_prompt="""
    You are an expert system diagnostics agent. Your role is to analyze system logs and metrics to identify issues and their high level root causes(including AWS resources such as ARNs,IDs etc. causing them).

When diagnosing system issues:
1. Start by gathering relevant logs (EC2, NGINX, DynamoDB)
2. Check CloudWatch metrics to understand resource utilization patterns
3. Correlate findings across and provide a fairly detailed but consize assessment with severity
4. Once the analysis is complete, in the end share the data sources or points(EC2s, tables etc.), based on which these insights were generated. 
"""
)

print("✅ Strandsエージェントが作成されました")
print(f"   エージェント: system_diagnostics_agent")
print(f"   モデル: {MODEL_ID}")
print(f"   ツール: 7 (EC2, NGINX, DynamoDB, CloudWatch)")

## 6. Strands エージェントをローカルでテスト

**目標:** モックデータを使用してエージェントの推論とツールオーケストレーションを検証します。

**アプローチ:** 診断クエリでエージェントを呼び出し、ツール呼び出しと推論をトレースします。

**重要な学習ポイント:** エージェントが問題を解決するためにツールをどのように選択し組み合わせるか。

In [ ]:
### 6.1: 診断クエリでエージェントをテスト

import asyncio

print("🧪 Strandsエージェントをローカルでテスト中\n")
print("=" * 70)

test_queries = [
    "What critical issues do you see in the system? Provide a summary with key data points."
]
diagnostics_agent_response=""
async def test_agent():
    responses=[]
    for i, query in enumerate(test_queries, 1):
        print(f"\n[クエリ {i}] {query}\n")
        
        try:
            # エージェントを実行（非同期呼び出し）
            response = await diagnostic_agent.invoke_async(query)
            print(f"エージェントレスポンス:\n{response}\n")
            responses.append(response.message['content'][0]['text'])
        except Exception as e:
            print(f"❌ エラー: {e}\n")
            responses.append(None)

    return responses
# Jupyter で非同期テストを実行
diagnostics_agent_response=await test_agent()

print("=" * 70)
print("✅ エージェントテスト完了")

## 診断分析を Agent Memory に書き込む

In [ ]:
nginx_instance_id = resources.get('nginx_instance_id')
app_instance_id = resources.get('app_instance_id')
crm_activities_table_name = resources.get('crm_activities_table_name')
crm_customers_table_name = resources.get('crm_customers_table_name')
crm_deals_table_name = resources.get('crm_deals_table_name')



memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

# メッセージからペイロードを構築

payload=[]   
payload.append({
            "conversational": {
                "content": {"text": "nginx EC2 instance id: " + nginx_instance_id + "."},
                "role": "ASSISTANT"
            }
        })
payload.append({
            "conversational": {
                "content": {"text": "application EC2 instance id: " + app_instance_id + "."},
                "role": "ASSISTANT"
            }
        })
payload.append({
            "conversational": {
                "content": {"text": "CRM Activities Table Name (DynamoDB): " + crm_activities_table_name+ "."},
                "role": "ASSISTANT"
            }
        })
payload.append({
            "conversational": {
                "content": {"text": "CRM Customers Table Name (DynamoDB): " + crm_customers_table_name+ "."},
                "role": "ASSISTANT"
            }
        })
payload.append({
            "conversational": {
                "content": {"text": "CRM Deals Table Name (DynamoDB): " + crm_deals_table_name+ "."},
                "role": "ASSISTANT"
            }
        })
payload.append({
            "conversational": {
                "content": {"text": "diagnostics agent analysis: " + str(diagnostics_agent_response)},
                "role": "ASSISTANT"
            }
        })
    
# 提供されたタイムスタンプまたは現在の UTC 時刻を使用
event_timestamp=datetime.datetime.now()
    
# リクエストパラメータを構築
params = {
        "memoryId": memory_id,
        "actorId": actor_id,
        "sessionId": memory_session_id,
        "eventTimestamp": event_timestamp,
        "payload": payload
    }
    
response = agent_memory_client.create_event(**params)    

# Agent Memory に追加されたイベントをリストして、書き込みが成功したことを確認
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }

response = agent_memory_client.list_events(**params)

for event in response.get("events", []):
    event_id = event.get('eventId')
    #print(f"\nEvent: {event_id}")
    
    # すべてのメッセージを取得
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            role = item['conversational']['role']
            print(f"  Message {i}: [{actor_id}] {text}")

## 7. Strands Lambda ハンドラーの作成

**目標:** AgentCore Gateway 呼び出し用に Strands エージェントをラップする Lambda ハンドラーを作成します。

**アプローチ:** Gateway イベントコンテキストを受信し、Strands エージェントを呼び出し、構造化されたレスポンスを返すハンドラーを構築します。

**重要な学習ポイント:** Strands エージェントを Lambda/Gateway インフラストラクチャとどのようにブリッジするか。

## 8. Strands エージェントを Lambda にデプロイ（ZIP ベース - VPC 互換）

**目標:** 実証済みの Strands エージェントコードを ZIP パッケージングを使用して Lambda にデプロイします。

**前提条件:** セクション 5-7 が完了しローカルでテスト済み。

In [ ]:
print(AWS_REGION)

In [ ]:
### 8.1: Lambda をデプロイ
!chmod +x lab_helpers/lab_02/deploy.sh
!lab_helpers/lab_02/deploy.sh

## 9. AgentCore Gateway の作成と Lambda ターゲットの登録

**目標:** デプロイされた Lambda 関数と連携するための Gateway インフラストラクチャをセットアップします。

**アプローチ:** Boto3 を使用して明示的に制御しながらステップバイステップで構築します。

**前提条件:** セクション 8 の Lambda デプロイが完了していること。

**重要な学習ポイント:** Gateway が IAM 認証を介して Lambda 関数間でツール呼び出しをどのようにオーケストレーションするか。

**アーキテクチャ:**
```
ユーザーリクエスト
    ↓
MCP クライアント（セクション 10）
    ↓（IAM 認証）
Gateway（セクション 9）
    ↓（Gateway サービスロール）
Lambda（セクション 8）
    ↓
Strands エージェント（セクション 5-7）
```

In [ ]:
### 9.0: Gateway サービスロールを作成

print("📋 Gatewayサービスロールをセットアップ中...\n")

from lab_helpers.lab_02.gateway_setup import create_gateway_service_role
from lab_helpers.config import AWS_REGION

# Gateway 用の IAM サービスロールを作成
gateway_role_config = create_gateway_service_role(region_name=AWS_REGION)

print(f"\n✅ Gatewayサービスロールの準備完了")
print(f"   ロールARN: {gateway_role_config['role_arn']}")
print(f"   権限: Lambda呼び出し + CloudWatchログ")

# 9.1 で使用するために保存
gateway_role_arn = gateway_role_config['role_arn']

In [ ]:
### 9.1: Gateway を作成

import boto3
from lab_helpers.config import AWS_REGION

# AgentCore クライアントを初期化
agentcore_client = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

print("📋 AgentCore Gatewayを作成中...")

try:
    # Parameter Store から Cognito 設定を取得
    from lab_helpers.parameter_store import get_parameter
    from lab_helpers.constants import PARAMETER_PATHS
    
    user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'], region_name=AWS_REGION)
    user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'], region_name=AWS_REGION)
    
    # ディスカバリー URL を構築（Lab 3 & 4 と同じパターン）
    discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"
    
    # CUSTOM_JWT 認証で Gateway を作成（Lab 3 & 4 と一致）
    gateway = agentcore_client.create_gateway(
        name="aiml301-diagnostics-gateway",
        roleArn=gateway_role_arn,  # Gateway サービスロール（9.0 から）
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",  # 呼び出し側は Cognito JWT を使用して Gateway を呼び出し
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": discovery_url,
                "allowedClients": [user_auth_client_id]
            }
        }
    )
    
    gateway_id = gateway['gatewayId']
    gateway_url = gateway['gatewayUrl']
    gateway_role_arn_actual = gateway['roleArn']
    
    print(f"✅ Gatewayが正常に作成されました")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {gateway_url}")
    print(f"   認証: CUSTOM_JWT (Cognito User Pool: {user_pool_id})")
    print(f"   許可されたクライアント: {user_auth_client_id}")
    print(f"   サービスロール: {gateway_role_arn_actual}")
    print(f"      (GatewayがLambdaターゲットを呼び出すために使用)")
    
    # Lab 5 用に Gateway 設定を Parameter Store に保存
    from lab_helpers.parameter_store import put_parameter
    put_parameter('/aiml301/lab-02/gateway-id', gateway_id, region_name=AWS_REGION)
    put_parameter('/aiml301/lab-02/gateway-url', gateway_url, region_name=AWS_REGION)
    print(f"✅ Gateway設定がParameter Storeに保存されました")
    
    # このノートブックで後で使用するために保存
    gateway_config = {
        'gateway_id': gateway_id,
        'gateway_url': gateway_url,
        'region': AWS_REGION
    }
    
except Exception as e:
    print(f"❌ エラー: {e}")
    raise

In [ ]:
print("📝 Strands診断エージェント用のツールスキーマを定義中...\n")

# Gateway は単一の高レベルツールを公開: "invoke_diagnostics_agent"
# このツールは自然言語クエリを受け取り、Lambda で実行されている Strands エージェントを呼び出す
# エージェントは内部で 4 つの診断ツール（EC2、NGINX、DynamoDB、CloudWatch）をオーケストレーション

tool_schema = [
    {
        "name": "invoke_diagnostics_agent",
        "description": "Invoke the diagnostics agent to analyze system logs and metrics. The agent will orchestrate multiple diagnostic tools (EC2 logs, NGINX logs, DynamoDB logs, CloudWatch metrics) to identify issues and root causes.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Natural language diagnostic query (e.g., 'What are the main issues?', 'Analyze CPU and memory utilization')"
                }
            },
            "required": ["query"]
        }
    }
]

print("✅ ツールスキーマが定義されました（1つの高レベルツール）")
print(f"   ツール: invoke_diagnostics_agent")
print(f"   目的: 自然言語による診断クエリ")
print(f"   内部: エージェントが4つの診断ツールをオーケストレーション (EC2, NGINX, DynamoDB, CloudWatch)")
print("\n📐 アーキテクチャ:")
print("   Gatewayツール: invoke_diagnostics_agent (自然言語インターフェース)")
print("        ↓")
print("   Lambdaハンドラー: クエリを受信し、Strandsエージェントを呼び出し")
print("        ↓")
print("   Strandsエージェント: ローカルツールをオーケストレーション")
print("        ├─ get_ec2_logs")
print("        ├─ get_nginx_logs")
print("        ├─ get_dynamodb_logs")
print("        └─ get_cloudwatch_metrics")

In [ ]:
import time
time.sleep(10)

In [ ]:
### 9.3: Lambda 関数を Gateway ターゲットとして登録

print("🔗 Lambdaをツールターゲットとして登録中...\n")

# Parameter Store から Lambda 関数 ARN を取得
ssm_client = boto3.client('ssm', region_name=AWS_REGION)
lambda_function_arn = ssm_client.get_parameter(
    Name='/aiml301/lab-02/lambda-function-arn'
)['Parameter']['Value']

print(f"Lambda ARN: {lambda_function_arn}\n")

try:
    # Lambda をツールターゲットとして登録
    target = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="strands-diagnostics-agent",
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_function_arn,
                    "toolSchema": {
                        "inlinePayload": tool_schema  # このターゲットで利用可能なツールを定義
                    }
                }
            }
        },
        credentialProviderConfigurations=[
            {
                "credentialProviderType": "GATEWAY_IAM_ROLE"  # Gateway の IAM ロールで Lambda を呼び出し
            }
        ]
    )
    
    target_id = target['targetId']
    
    print("✅ Lambdaターゲットが正常に登録されました")
    print(f"   ターゲットID: {target_id}")
    print(f"   ターゲット名: strands-diagnostics-agent")
    print(f"   Lambda ARN: {lambda_function_arn}")
    print(f"   ツール: 4 (get_ec2_logs, get_nginx_logs, get_dynamodb_logs, get_cloudwatch_metrics)")
    print(f"   認証情報: GATEWAY_IAM_ROLE (GatewayサービスロールでLambdaを呼び出し)")
    
    # ターゲット情報で Gateway 設定を更新
    gateway_config['target_id'] = target_id
    gateway_config['lambda_arn'] = lambda_function_arn
    
    print("\n📊 Gateway設定サマリー:")
    print(f"   Gateway ID: {gateway_config['gateway_id']}")
    print(f"   Gateway URL: {gateway_config['gateway_url']}")
    print(f"   ターゲットID: {gateway_config['target_id']}")
    print(f"   リージョン: {gateway_config['region']}")
    
except Exception as e:
    print(f"❌ ターゲット登録エラー: {e}")
    raise

## 10. Cognito JWT 認証で Gateway をテスト

**目標:** Cognito JWT トークンを使用して Gateway をテストします（Lab 3、4、5 と同じパターン）。

**アプローチ:**
1. Cognito で認証して JWT トークンを取得
2. JWT Bearer トークンでカスタム HTTP MCP クライアントを使用
3. エンドツーエンドフローをテスト

**重要な学習ポイント:** 一貫性のためにすべての Lab で使用される JWT 認証パターン。

In [ ]:
# Cognito 設定を取得
print("🔐 Cognito設定を取得中...")
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'], region_name=AWS_REGION)
user_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'], region_name=AWS_REGION)
test_username = get_parameter(PARAMETER_PATHS['cognito']['test_user_email'], region_name=AWS_REGION)
test_password = get_parameter(PARAMETER_PATHS['cognito']['test_user_password'], region_name=AWS_REGION)

print(f"  ✓ User Pool: {user_pool_id}")
print(f"  ✓ Client ID: {user_client_id}")
print(f"  ✓ ユーザー名: {test_username}")

# Cognito で認証
print("\n🔑 Cognitoで認証中...")
cognito = boto3.client('cognito-idp', region_name=AWS_REGION)

response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': test_username,
        'PASSWORD': test_password
    }
)

access_token = response['AuthenticationResult']['AccessToken']
id_token = response['AuthenticationResult']['IdToken']
expires_in = response['AuthenticationResult']['ExpiresIn']

print(f"  ✅ 認証に成功しました！")
print(f"  ✓ トークンタイプ: Bearer")
print(f"  ✓ 有効期限: {expires_in}秒 ({expires_in // 60}分)")
print(f"  ✓ アクセストークン (最初の50文字): {access_token[:50]}...")
print(f"\n📋 JWTトークンを取得し、Gateway認証の準備ができました")

In [ ]:
### 10.2: JWT トークンで MCP クライアントを初期化

from lab_helpers.lab_02.mcp_client import MCPClient

print("🔗 Gateway用のMCPクライアントを初期化中\n")

# セクション 9 から Gateway URL を取得
gateway_url = gateway_config['gateway_url']

# JWT Bearer トークンで MCP クライアントを作成
mcp_client = MCPClient(gateway_url, access_token)

# MCP セッションを初期化
mcp_client.initialize()

print(f"\n✅ MCPクライアントの準備完了")
print(f"   Gateway URL: {gateway_url}")
print(f"   認証: JWT Bearerトークン")
print(f"   セッション: 初期化済み")
print("\n📝 クライアントはMCPプロトコル経由でツールを呼び出す準備ができました")

In [ ]:
### 10.3: Gateway で利用可能なツールを一覧表示

print("📋 Gatewayで利用可能なツールを一覧表示中\n")

# MCP プロトコルでツールを一覧表示
tools = mcp_client.list_tools()

print(f"\n✅ Gatewayは{len(tools)}個のツールで準備完了です")

### 10.3: Gateway 経由でツール呼び出しをテスト

In [ ]:
### 10.4: Gateway 経由でツール呼び出しをテスト

print("🧪 Gateway MCP経由でツール呼び出しをテスト中\n")
print("=" * 70)

# Gateway はツール名にターゲットをプレフィックスとして付加: "strands-diagnostics-agent___<tool_name>"
agent_tool_full = "strands-diagnostics-agent___invoke_diagnostics_agent"
test_query = "Provide a summary of critical issues with key data points and resource details."
test_args = {"query": test_query}

print(f"\n📤 テストクエリ: {test_query}\n")

try:
    # MCP クライアントを使用してツールを呼び出し
    result = mcp_client.call_tool(agent_tool_full, test_args)
    
    # レスポンスを抽出して表示
    if 'content' in result:
        print("\n" + "=" * 70)
        print("📥 エージェントレスポンス:")
        print("=" * 70)
        
        for item in result['content']:
            if item.get('type') == 'text':
                text = item.get('text', '')
                # 可能であれば JSON をパース
                try:
                    import json
                    parsed = json.loads(text)
                    if 'response' in parsed:
                        print(f"\n{parsed['response']}")
                    else:
                        print(f"\n{json.dumps(parsed, indent=2)}")
                except json.JSONDecodeError:
                    print(f"\n{text}")
    
    print("\n" + "=" * 70)
    print("✅ ツール呼び出しテスト完了")
    
except Exception as e:
    print(f"\n❌ エラー: {e}")
    import traceback
    traceback.print_exc()
    print("\n" + "=" * 70)

## 11. Lab 02 リソースのクリーンアップ

**目的:** Lab 02 で作成されたすべてのリソースを削除し、新たに開始できるようにします。

**注意:** これにより以下が削除されます:
- AgentCore Gateway
- Lambda 関数
- ECR リポジトリ
- IAM ロール
- Parameter Store エントリ
- CloudWatch ログ

Lab 02 を完了した場合、または最初からやり直したい場合にこのセクションを実行してください。

In [ ]:
### 11.1: Lab 02 のすべてのリソースをクリーンアップ
# 次に Lab-03 を実行する予定がない場合のみ実行

from lab_helpers.lab_02.cleanup import cleanup_lab_02
from lab_helpers.config import AWS_REGION

# クリーンアップを実行
#cleanup_lab_02(region_name=AWS_REGION)

## まとめ: Lab 2 - 診断エージェントアーキテクチャ

✅ **完了:**
1. ✓ ヘルパーツール - モックデータサポート付きの CloudWatch ログとメトリクス取得
2. ✓ IAM 設定 - 必要な権限を持つ Lambda 実行ロール
3. ✓ エージェントアーキテクチャ - 診断用の Strands エージェント

**ワークフローまとめ:**
```
ユーザーリクエスト
    ↓
AgentCore Gateway
    ↓
Lambda 関数（ECR コンテナ）
    ↓
Strands エージェント + ツール
    ├─ fetch_ec2_logs() [モック/ライブ]
    ├─ fetch_nginx_logs() [モック/ライブ]
    ├─ fetch_dynamodb_logs() [モック/ライブ]
    └─ fetch_metrics() [モック/ライブ]
    ↓
分析出力
    ↓
Gateway に戻る
```

**次へ: Lab 3 - 修復エージェント** (`Lab-03-remediation-agent.ipynb`)
- 修復アクションの承認ワークフロー
- 安全なスクリプト実行のための Code Interpreter
- 診断エージェントの検出結果との統合